## Instagram Hashtag Analysis In Python<br>
<br>
Extract description(hashtags) of a post from Instagram link.<br>
    driver.get('https://www.instagram.com/p/Bakrxbnlx49/')

In [ ]:
pip install pandas

In [5]:
"""
Created on Mon Sep  9 15:29:48 2019
Instagram Hashtag Analysis In Python
@author: yoko
"""

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import time        #note this line
#from selenium.webdriver.support.ui import WebDriverWait

driver = webdriver.Chrome()
 
# Extract description of a post from Instagram link
#driver.get('https://www.instagram.com/p/BiRnjDsFKzl/')
driver.get('https://www.instagram.com/p/Bakrxbnlx49/')
soup = BeautifulSoup(driver.page_source,"lxml")
desc = " "
 
for item in soup.findAll('a'):
    desc= desc + " " + str(item.string)
 
# Extract tag list from Instagram post description
taglist = desc.split()
taglist = [x for x in taglist if x.startswith('#')]
index = 0
while index < len(taglist):
    taglist[index] = taglist[index].strip('#')
    index += 1
 
# (OR) Copy-paste your tag list manually here
#taglist = ['art', 'instaart', 'iblackwork']
 
print(taglist)

['japaneseversion', 'ワイン', 'フランスワイン', 'ボルドー', 'ワイン醸造学']


Extract total number of posts in each hashtags.<b>
Extract all post links from 'explore tags' page.
Needed to extract post frequency of recent posts.
Extract the posting time of 1st and 9th most recent post for a tag.<br>
Calculate time difference between posts for obtaining posting frequency.

In [6]:
# Define dataframe to store hashtag information
tag_df  = pd.DataFrame(columns = ['Hashtag', 'Number of Posts', 'Posting Freq (mins)'])
tag_dataframe = []
 
# Loop over each hashtag to extract information
for tag in taglist:
     
    driver.get('https://www.instagram.com/explore/tags/'+str(tag))

### for non english problem
    time.sleep(12)        #note this as well moreover it should be after get method 
#    post = WebDriverWait(driver, 10).until(
#            lambda driver: driver.find_element_by_class_name('g47SY')
#            )

###

    soup = BeautifulSoup(driver.page_source,"lxml")
     
    # Extract current hashtag name
    tagname = tag
    # Extract total number of posts in this hashtag
    # NOTE: Class name may change in the website code
    # Get the latest class name by inspecting web code

    if soup.find("span", class_="g47SY") is None:
        nposts = "1"
        print(tagname + ' : ' + nposts)
    else:
        nposts = soup.find("span", class_="g47SY").text
#    nposts = soup.find('span', {'class': 'g47SY'}).text 
        print(tagname + ' : ' + nposts)
     
    # Extract all post links from 'explore tags' page
    # Needed to extract post frequency of recent posts
    myli = []
    for a in soup.find_all('a', href=True):
        myli.append(a['href'])
 
    # Keep link of only 1st and 9th most recent post 
    newmyli = [x for x in myli if x.startswith('/p/')]
    del newmyli[:9]
    del newmyli[9:]
    del newmyli[1:8]
 
    timediff = []
 
    # Extract the posting time of 1st and 9th most recent post for a tag
    for j in range(len(newmyli)):
        driver.get('https://www.instagram.com'+str(newmyli[j]))
        soup = BeautifulSoup(driver.page_source,"lxml")
 
        for i in soup.findAll('time'):
            if i.has_attr('datetime'):
                timediff.append(i['datetime'])
                #print(i['datetime'])
 
    # Calculate time difference between posts
    # For obtaining posting frequency
    if timediff==[]:
        pfreq=0
    else:
        datetimeFormat = '%Y-%m-%dT%H:%M:%S.%fZ'
        diff = datetime.datetime.strptime(timediff[0], datetimeFormat) - datetime.datetime.strptime(timediff[1], datetimeFormat)
        pfreq= int(diff.total_seconds()/(9*60))
    print(pfreq)
    
    # Add hashtag info to dataframe
    tag_df.loc[len(tag_df)] = [tagname, nposts, pfreq]
    #add_hashtag_tag_df = pd.DataFrame([tagname, nposts, pfreq])
    #tag_df.append(add_hashtag_tag_df)
#    tag_dataframe.append([tagname, nposts, pfreq])
        
driver.quit()

# Check the final dataframe
print(tag_df)
 
# CSV output for hashtag analysis
#tag_df.to_csv('hashtag_list.csv')

japaneseversion : 15,240
257
ワイン : 3,338,630
0
フランスワイン : 42,425
0
ボルドー : 305,488
0
ワイン醸造学 : 1
0
           Hashtag Number of Posts Posting Freq (mins)
0  japaneseversion          15,240                 257
1              ワイン       3,338,630                   0
2          フランスワイン          42,425                   0
3             ボルドー         305,488                   0
4           ワイン醸造学               1                   0


In [7]:
!jupyter nbconvert --to html Instagram_hashtag_analysis.ipynb

[NbConvertApp] Converting notebook Instagram_hashtag_analysis.ipynb to html
[NbConvertApp] Writing 289811 bytes to Instagram_hashtag_analysis.html
